In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv("covid_toy.csv")

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=["has_covid"]),df["has_covid"],test_size=0.2)

In [14]:
X_train

,age,gender,fever,cough,city
70,68,Female,101.0,Strong,Delhi
99,10,Female,98.0,Strong,Kolkata
82,24,Male,98.0,Mild,Kolkata
6,14,Male,101.0,Strong,Bangalore
61,81,Female,98.0,Strong,Mumbai
...,...,...,...,...,...
50,19,Male,101.0,Mild,Delhi
41,82,Male,NaN,Mild,Kolkata
46,19,Female,101.0,Mild,Mumbai
89,46,Male,103.0,Strong,Bangalore


1.Normal method

In [16]:
#adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[["fever"]])

#also the test data
X_test_fever = si.fit_transform(X_test[["fever"]])

X_train_fever.shape

(80, 1)

In [ ]:
#ordinal encoding -> cough
oe = OrdinalEncoder(categories=[["Mild","Strong"]])

X_train_cough = oe.fit_transform(X_train[["cough"]])

#also the test data
X_test_cough = oe.fit_transform(X_test[["cough"]])

X_train_cough.shape

(80, 1)

In [25]:
#OneHotEncoding -> gender city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[["gender","city"]])

#also the test data
X_test_gender_city  = ohe.fit_transform(X_test[["gender","city"]])

X_train_gender_city.shape

(80, 4)

In [26]:
#Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

#also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
#Also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

Column transformer method

In [28]:
from sklearn.compose import ColumnTransformer

In [39]:
transformer = ColumnTransformer(transformers=[
    ("tnf1",SimpleImputer(),["fever"]),
    ("tnf2",OrdinalEncoder(categories=[["Mild","Strong"]]),["cough"]),
    ("tnf3",OneHotEncoder(sparse_output=False,drop="first"),["gender","city"])
],remainder="passthrough")

In [40]:
transformer.fit_transform(X_train)

array([[101.   ,   1.   ,   0.   ,   1.   ,   0.   ,   0.   ,  68.   ],
       [ 98.   ,   1.   ,   0.   ,   0.   ,   1.   ,   0.   ,  10.   ],
       [ 98.   ,   0.   ,   1.   ,   0.   ,   1.   ,   0.   ,  24.   ],
       [101.   ,   1.   ,   1.   ,   0.   ,   0.   ,   0.   ,  14.   ],
       [ 98.   ,   1.   ,   0.   ,   0.   ,   0.   ,   1.   ,  81.   ],
       [104.   ,   0.   ,   1.   ,   0.   ,   0.   ,   0.   ,  51.   ],
       [ 98.   ,   0.   ,   0.   ,   1.   ,   0.   ,   0.   ,  80.   ],
       [ 98.   ,   0.   ,   0.   ,   0.   ,   1.   ,   0.   ,  31.   ],
       [100.875,   0.   ,   1.   ,   0.   ,   0.   ,   1.   ,  23.   ],
       [100.875,   0.   ,   0.   ,   1.   ,   0.   ,   0.   ,  75.   ],
       [ 99.   ,   1.   ,   0.   ,   0.   ,   1.   ,   0.   ,  25.   ],
       [ 98.   ,   0.   ,   0.   ,   0.   ,   0.   ,   1.   ,  65.   ],
       [100.875,   1.   ,   0.   ,   0.   ,   0.   ,   0.   ,  42.   ],
       [104.   ,   1.   ,   0.   ,   1.   ,   0.   ,   0.   ,  3

In [41]:
transformer.fit_transform(X_train).shape

(80, 7)

In [43]:
transformer.fit_transform(X_test)

array([[101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   8.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          1.        ,  11.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   6.        ],
       [100.        ,   1.        ,   0.        ,   0.        ,
          0.        ,  19.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  64.        ],
       [102.        ,   1.        ,   0.        ,   0.        ,
          0.        ,  24.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,  27.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,  80.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          0.        ,  73.        ],
       [100.72222222,   1.        ,   1.        ,   0.        ,
          1.        ,  71. 

In [44]:
transformer.fit_transform(X_test).shape

(20, 6)